<font size = 5><center>Midterm Course Outcome Output</center></font><br>
<font size = 5><center><strong>Data Wrangling, Data Analysis, and Data Visualization</strong></center></font><br>
<font size = 5><center>(25k IMDb Movie Dataset)</center></font><br>
<center><font size = 4>John Leo D. Echevaria (A22-34233)</font><br/>College of Computing and Multimedia Studies<br/>Manuel S. Enverga University Foundation</center>

In [164]:
import pandas as pd
import hvplot.pandas
import holoviews as hv

In [165]:
df = pd.read_csv('25k IMDb movie Dataset.csv')
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


## Data Wrangling

In [166]:
df.columns = [col.title() for col in df.columns]

In [167]:
# Fixing the typo on the Genre column
df.rename(columns={'Generes':'Genres'},inplace=True)

In [168]:
# Removing unnecessary columns
df.drop(columns=['Overview','Plot Kyeword','Path'],inplace=True)

In [169]:
# Removing the non-digit characters in the year column
df['Year'] = df['Year'].str.replace(r'[^\d]', '',regex=True)

In [170]:
# Converting the year column datas into integer and removing rows without datas or NaN.
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df = df.dropna(subset=['Year'])
df.Year = df.Year.astype(int)

In [171]:
# Converting the rating column into numeric values
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

In [172]:
# Converting the user rating column into numeric value and multiplying by thousands if 'k' and millions if 'm'
def convert_rating_to_number(rating):
    if 'k' in rating:
        return int(float(rating.replace('k', ''))) * 1000
    elif 'm' in rating:
        return int(float(rating.replace('m', ''))) * 1000000
    else:
        return int(float(rating))

df['User Rating'] = df['User Rating'].str.lower().apply(convert_rating_to_number)

In [173]:
# There are datas beyond 2023 so we'll remove it.
df = df.drop(df[df['Year'] > 2022].index)

In [174]:
# There are movies included that are not-released.
df = df.drop(df[df['Run Time'] == 'not-released'].index)

In [175]:
# Dropping Run Time because it seems to have mixed values of data.
df.drop(columns=['Run Time'],inplace=True)

In [176]:
df_exploded_genres = df
df_exploded_casts = df

In [177]:
# Exploding the list of genres on movies, and assigning it to a new dataframe. (To be used later)
df_exploded_genres['Genres'] = df_exploded_genres['Genres'].astype(str)
df_exploded_genres['Genres'] = df_exploded_genres['Genres'].str.strip('[]')
df_exploded_genres['Genres'] = df_exploded_genres['Genres'].str.split(', ')
df_exploded_genres = df_exploded_genres.explode('Genres')

In [178]:
# Exploding the list of top 5 casts on movies, and assigning it to a new dataframe. (To be used later)
df_exploded_casts['Top 5 Casts'] = df_exploded_casts['Top 5 Casts'].astype(str)
df_exploded_casts['Top 5 Casts'] = df_exploded_casts['Top 5 Casts'].str.strip('[]')
df_exploded_casts['Top 5 Casts'] = df_exploded_casts['Top 5 Casts'].str.split(', ')
df_exploded_casts = df_exploded_casts.explode('Top 5 Casts')

## Data Analysis and Visualization

### How does the number of movies released each year vary over time?

In [179]:
movie_per_year = df.groupby(['Year'])['Movie Title'].count()
movie_per_year.hvplot(title='Number of Movies Released Over Time (1906-2022)',
                      ylabel='Number of Movies Released', xlabel='Year of Release',
                      line_width=4, fontsize={'title':18, 'ylabel':12, 'xlabel':12})

:Curve   [Year]   (Movie Title)

### How did the advancement in film technology affect the number of films produced over the year?

Sound technology: The first synchronized sound feature film, "The Jazz Singer," was released in 1927, marking the beginning of the sound era in cinema.

Color technology: Technicolor, the first widely used color film process, was introduced in 1932 with the release of "Flowers and Trees," a Walt Disney animated short.

Special effects: Advances in special effects technology began in the early 1950s with films such as "The Day the Earth Stood Still" (1951) and "Forbidden Planet" (1956), which used innovative techniques to create realistic-looking alien worlds and creatures.

Digital technology: The first digital camera specifically designed for movie production, the Sony HDW-F900, was introduced in 1998. Since then, digital filmmaking has become increasingly common, with many filmmakers choosing to shoot and edit their movies entirely on digital platforms.

In [180]:
# Creating lines to indicate division of film technology eras.
vline_sound = hv.VLine(1927).options(color='grey',alpha=0.6).opts(line_dash='dashed')
vline_color = hv.VLine(1932).options(color='grey',alpha=0.6).opts(line_dash='dashed')
vline_sfx = hv.VLine(1951).options(color='grey',alpha=0.6).opts(line_dash='dashed')
vline_digital = hv.VLine(1998).options(color='grey',alpha=0.6).opts(line_dash='dashed')

In [181]:
# Overlay the plot, lines, and texts.
movie_per_year.hvplot(title='Number of Movies Released Over Time and the Advances in Film Technology (1906-2022)',
                      ylabel='Number of Movies Released', xlabel='Year of Release',
                      height=500, width=1000, line_width=4, fontsize={'title':16, 'ylabel':12, 'xlabel':12}) * \
                      vline_sound * hv.Text(1926,17,"Start of Sound Technology",halign='left',rotation=90) * \
                      vline_color * hv.Text(1931,30,"Start of Color Technology",halign='left',rotation=90) * \
                      vline_sfx * hv.Text(1950,60,"Start of SFX Technology",halign='left',rotation=90) * \
                      vline_digital * hv.Text(1997,240,"Start of Digital Technology",halign='left',rotation=90)


:Overlay
   .Curve.Movie_Title :Curve   [Year]   (Movie Title)
   .VLine.I           :VLine   [x,y]
   .Text.I            :Text   [x,y]
   .VLine.II          :VLine   [x,y]
   .Text.II           :Text   [x,y]
   .VLine.III         :VLine   [x,y]
   .Text.III          :Text   [x,y]
   .VLine.IV          :VLine   [x,y]
   .Text.IV           :Text   [x,y]

### What Are The Trend For Each Genre Over Time?

In [197]:
genre_trend = df_exploded_genres.groupby(['Genres','Year'])['Movie Title'].count()

genre_trend = genre_trend.hvplot('Year', groupby='Genres', dynamic=False, height=500, width=1000) * \
                      vline_sound * hv.Text(1926,0,"Sound Technology",halign='left',rotation=90) * \
                      vline_color * hv.Text(1931,0,"Color Technology",halign='left',rotation=90) * \
                      vline_sfx * hv.Text(1950,0,"SFX Technology",halign='left',rotation=90) * \
                      vline_digital * hv.Text(1997,0,"Digital Technology",halign='left',rotation=90)
genre_trend

:HoloMap   [Genres]
   :Overlay
      .Curve.Movie_Title :Curve   [Year]   (Movie Title)
      .VLine.I           :VLine   [x,y]
      .Text.I            :Text   [x,y]
      .VLine.II          :VLine   [x,y]
      .Text.II           :Text   [x,y]
      .VLine.III         :VLine   [x,y]
      .Text.III          :Text   [x,y]
      .VLine.IV          :VLine   [x,y]
      .Text.IV           :Text   [x,y]

In [208]:
genre_trend["'Drama'"].opts(title='Number of Drama Films Produced in Relation to Film Technology',ylabel='Number Of Films')

:Overlay
   .Curve.Movie_Title :Curve   [Year]   (Movie Title)
   .VLine.I           :VLine   [x,y]
   .Text.I            :Text   [x,y]
   .VLine.II          :VLine   [x,y]
   .Text.II           :Text   [x,y]
   .VLine.III         :VLine   [x,y]
   .Text.III          :Text   [x,y]
   .VLine.IV          :VLine   [x,y]
   .Text.IV           :Text   [x,y]

In [209]:
genre_trend["'Sci-Fi'"].opts(title='Number of Sci-Fi Films Produced in Relation to Film Technology',ylabel='Number Of Films')

:Overlay
   .Curve.Movie_Title :Curve   [Year]   (Movie Title)
   .VLine.I           :VLine   [x,y]
   .Text.I            :Text   [x,y]
   .VLine.II          :VLine   [x,y]
   .Text.II           :Text   [x,y]
   .VLine.III         :VLine   [x,y]
   .Text.III          :Text   [x,y]
   .VLine.IV          :VLine   [x,y]
   .Text.IV           :Text   [x,y]

### Which genre were commonly used on films? (Top 10)

In [183]:
# We will be needing to explode the genres because it is in a list form.
common_genres = df['Genres'].explode().value_counts().sort_values(ascending=True).tail(10)

common_genres.hvplot.bar(title='Top 10 Most Common Genres of Movie',
                          ylabel='Number of Movies', xlabel='Genres of Film')

:Bars   [index]   (Genres)

### Top 10 Genres of Movie Based on Average Ratings

In [184]:
popular_genres = df_exploded_genres.groupby(['Genres'])['Rating'].mean().sort_values(ascending=True).tail(10)
popular_genres.hvplot.barh(title='Top 10 Genres Based on Average Ratings')

:Bars   [Genres]   (Rating)

### Which Director Has Directed the Most Number of Movies? (Top 10)

In [185]:
director = df.groupby(['Director'])['Movie Title'].count().sort_values(ascending=True).tail(10)

director.hvplot.barh(title='Top 10 Directors Based on the Number of Movies Directed',
                     ylabel='Number of Movies',xlabel='Name of Director')

:Bars   [Director]   (Movie Title)

### Which Director Has The Highest Rating on The Films That They Have Produced?

In [186]:
# We will be filtering the data frame to only include films that has over 1 Million reviews in order to ensure the quality of the films.
director = df[df['User Rating']>1000000].groupby(['Director'])['Rating'].mean().sort_values(ascending=True).dropna()

director.hvplot.barh(title='Top Directors with Highest Average Rating on Their Films (1M+ Ratings)')

:Bars   [Director]   (Rating)

### Which Writers Has Most Number of Writing Credits? (Top 10)

In [187]:
director = df.groupby(['Writer'])['Movie Title'].count().sort_values(ascending=True).tail(10)

director.hvplot.barh(title='Top 10 Writers Based on the Number of Writing Credits',
                     ylabel='Numbber of Writing Credits',xlabel='Name of Writer')

:Bars   [Writer]   (Movie Title)

### Which Writer Has The Highest Rating on The Films That They Have Written?

In [188]:
# We will be filtering the data frame to only include films that has over 1 Million reviews in order to ensure the quality of the films.
writer = df[df['User Rating']>1000000].groupby(['Writer'])['Rating'].mean().sort_values(ascending=True).dropna()

writer.hvplot.barh(title='Top Writer with Highest Average Rating on Their Films (1M+ Ratings)')

:Bars   [Writer]   (Rating)

### Which Actors/Actresses Appeared In Most Number of Films (Top 10)

In [189]:
top_actors = df_exploded_casts.groupby(['Top 5 Casts'])['Movie Title'].count().sort_values().tail(10)
top_actors.hvplot.barh(title='Top 10 Actors/Actresses with the Most Film Appearances',
                       ylabel='Number of Movie Appearances',xlabel='Name')

:Bars   [Top 5 Casts]   (Movie Title)